In [ ]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

output_version = "20241229"
outdir = "/media/hieunguyen/HNSD_mini/outdir"
dataset_name = "LOD"
mode = "all"
PROJECT = "combine_ctcandi_ichorcna"

input_cancer_class = "pan_cancer"

path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
path_to_09_output = os.path.join(path_to_main_output, f"09_output_{mode}", input_cancer_class)
os.system(f"mkdir -p {path_to_09_output}")

metadata = pd.read_csv(f"./metadata/{dataset_name}.csv")
metadata = metadata[metadata["Sample"].duplicated() == False]